In [ ]:
import pandas as pd
import numpy as np

In [22]:
def process_transactions(sf_file, robo_file, start_date, end_date):
    """
    Process transactions from Salesforce and Robo files, filter by date range, and merge the results.

    Parameters:
        sf_file (str): Path to the Salesforce Excel file.
        robo_file (str): Path to the Robo CSV file.
        start_date (str): Start date for filtering (format: 'YYYY-MM-DD HH:MM:SS').
        end_date (str): End date for filtering (format: 'YYYY-MM-DD HH:MM:SS').

    Returns:
        dict: A dictionary containing the filtered DataFrames and the merged result:
            - 'sf': Filtered Salesforce DataFrame.
            - 'robo': Filtered Robo DataFrame.
            - 'merged': Merged DataFrame.
            - 'only_in_robo': Rows only in Robo.
            - 'only_in_sf': Rows only in Salesforce.
    """
    # load the files
    sf = pd.read_excel(sf_file)
    robo = pd.read_excel(robo_file)

    # Convert 'Date' in robo_file (ISO format)
    robo['Payment Date'] = pd.to_datetime(
        robo['Payment Date'],
        format='%Y-%m-%d %H:%M:%S',
        errors='coerce'
    )

    # Convert 'Finish Date' in sf (US format with AM/PM)
    sf['Finish Date'] = pd.to_datetime(sf['Finish Date'], format='%m/%d/%Y, %I:%M %p')

    # Optionally, if you want to display or export in a specific string format (e.g., yyyy-mm-dd HH:MM:SS)
    robo['Date_str'] = robo['Payment Date'].dt.strftime('%Y-%m-%d %H:%M:%S')
    sf['Finish Date_str'] = sf['Finish Date'].dt.strftime('%Y-%m-%d %H:%M:%S')

    # convert start_date and end_date to datetime
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # filter based on the date range
    sf = sf[(sf['Finish Date'] >= start_date) & (sf['Finish Date'] <= end_date)]
    robo = robo[(robo['Payment Date'] >= start_date) & (robo['Payment Date'] <= end_date)]

    robo['Transaction Name'] = robo['Transaction Name'].apply(lambda x: str(int(float(x))) if pd.notnull(x) else None)
    sf['Roboticket ID'] = sf['Roboticket ID'].apply(lambda x: str(int(float(x))) if pd.notnull(x) else None)
    sf['Transaction: Transaction Name'] = sf['Transaction: Transaction Name'].apply(lambda x: str(x).strip() if pd.notnull(x) else None)

    # print shapes
    print('Shape of sf:', sf.shape)
    print('Shape of robo:', robo.shape)

    # merge the DataFrames
    merged = pd.merge(sf, robo, how='outer', left_on='Transaction: Transaction Name', right_on='Transaction Name', indicator=True)

    print("Merged columns:", merged.columns.tolist())
    merged = merged.rename(columns={
    'Roboticket ID_x': 'Roboticket ID SF',
    'Roboticket ID_y': 'Roboticket ID Robo',
    'Finish Date_x': 'Finish Date SF',
    'Finish Date_y': 'Finish Date Robo'
    })


    merged = merged[['Roboticket ID SF',
                    'Transaction: Transaction Name',
                    'Finish Date SF',
                    'Payment Date',
                    'Transaction Name',
                    '_merge']]

    # Rename the values in the '_merge' column
    merged['_merge'] = merged['_merge'].replace({
        'left_only': 'exists_in_sf',
        'right_only': 'exists_in_robo'
    })

    # separate rows only in Robo and only in Salesforce
    only_in_robo = merged[merged['_merge'] == 'exists_in_robo']
    only_in_sf = merged[merged['_merge'] == 'exists_in_sf']
    both = merged[merged['_merge'] == 'both']

    # Save results to an Excel file with separate sheets
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        both.to_excel(writer, sheet_name='Both', index=False)
        only_in_sf.to_excel(writer, sheet_name='Exists_in_SF', index=False)
        only_in_robo.to_excel(writer, sheet_name='Exists_in_Robo', index=False)

    print(f"Results saved to {output_file}")

    # return results as a dictionary
    return {
        'sf': sf,
        'robo': robo,
        'merged': merged,
        'only_in_robo': only_in_robo,
        'only_in_sf': only_in_sf,
        'both': both
    }

In [ ]:
sf_file = 'All Time Transactions Report-2025-07-02-03-55-47.xlsx'
robo_file = 'transactions_modified.xlsx'
start_date = '2014-01-01 00:00:00'
end_date = '2025-07-02 23:59:59'
output_file = 'Processed_Transactions.xlsx'

results = process_transactions(sf_file, robo_file, start_date, end_date)

# Check results
print('Shape of merged:', results["merged"].shape)

In [24]:
print('Shape of only_in_robo:', only_in_robo.shape)
only_in_robo.to_excel('Only_in_Robo.xlsx', index=False)
only_in_robo

Shape of only_in_robo: (0, 6)


,Roboticket ID SF,Transaction: Transaction Name,Finish Date SF,Payment Date,Transaction Name,_merge


In [25]:
# Gaps: 
# 1. Time Gap: transactions that exist in robo on one date but exist in sf on another date means we can't
#  get the right transaction number

# 2. Return Transactions: those transactions receive a new transaction number in sf, means we can't follow a transaction that created
# and returend cause we got a new transaction number

# 3. Transactions without coOwner?

# 4. Lite Users: some of the lite users does not exist in sf. for example the players profile that we created in robo system

# 5. Membership cards that we generate tickets for everygame don't exist in sf - means when we generate tickets for them
# there is not transaction

# 6. Voucher Transactions: this transaction - 3551108 is only in robo and belongs to a voucher created in robo system, 
# was done by shoham hapoel


In [26]:
print('Shape of only_in_sf:', only_in_sf.shape)
only_in_sf

Shape of only_in_sf: (62335, 6)


,Roboticket ID SF,Transaction: Transaction Name,Finish Date SF,Payment Date,Transaction Name,_merge
0,3263347,3263347,2024-05-23 17:00:00,NaT,NaN,exists_in_sf
1,3263379,3263379,2024-05-04 17:00:00,NaT,NaN,exists_in_sf
2,3263380,3263380,2024-05-04 17:00:00,NaT,NaN,exists_in_sf
3,3263381,3263381,2024-05-04 17:00:00,NaT,NaN,exists_in_sf
4,3263382,3263382,2024-05-04 17:00:00,NaT,NaN,exists_in_sf
...,...,...,...,...,...,...
64687,3747717,3747717,2025-06-29 14:33:00,NaT,NaN,exists_in_sf
64688,3747718,3747718,2025-06-29 14:32:00,NaT,NaN,exists_in_sf
64689,3747720,3747720,2025-06-29 14:44:00,NaT,NaN,exists_in_sf
64690,3747726,3747726,2025-06-29 16:09:00,NaT,NaN,exists_in_sf


In [89]:
# Gaps:

# 1. Time Gap: this transaction in sf - 3573843 happend on 12/01/2024 but the product which is the ticket was created on 12/2/2024



In [90]:
print('Shape of both:', both.shape)
both

Shape of both: (69181, 6)


,Roboticket ID,Transaction: Transaction Name,Finish Date,Date,Transaction identifier,_merge
0,3263347,3263347,2024-05-23 17:00:00,2024-05-24 13:02:45,3263347,both
1,3263379,3263379,2024-05-04 17:00:00,2024-05-05 00:00:00,3263379,both
2,3263380,3263380,2024-05-04 17:00:00,2024-05-05 00:00:00,3263380,both
3,3263381,3263381,2024-05-04 17:00:00,2024-05-05 00:00:00,3263381,both
4,3263382,3263382,2024-05-04 17:00:00,2024-05-05 00:00:00,3263382,both
...,...,...,...,...,...,...
69394,3747191,3747191,2025-06-26 09:52:00,2025-06-26 16:52:53,3747191,both
69395,3747204,3747204,2025-06-26 14:52:00,2025-06-26 21:52:35,3747204,both
69396,3747226,3747226,2025-06-27 02:08:00,2025-06-27 09:08:06,3747226,both
69397,3747237,3747237,2025-06-27 08:30:00,2025-06-27 15:30:11,3747237,both


## Check transactions on robo file that don't exists in sf:

In [135]:
check = pd.read_excel('check.xlsx')
robo = pd.read_csv('TransactionReportCsv_From_2015-01-01_To_2025-06-29.csv')
users = pd.read_csv('UserCreatedReport_2025-05-21 10_07.csv')

/var/folders/nt/lqq9984n7p3fpfvd92swmw280000gn/T/ipykernel_13120/2439955604.py:2: DtypeWarning: Columns (2,10,13,14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  robo = pd.read_csv('TransactionReportCsv_From_2015-01-01_To_2025-06-29.csv')
/var/folders/nt/lqq9984n7p3fpfvd92swmw280000gn/T/ipykernel_13120/2439955604.py:3: DtypeWarning: Columns (14,15,19,24,25,28) have mixed types. Specify dtype option on import or set low_memory=False.
  users = pd.read_csv('UserCreatedReport_2025-05-21 10_07.csv')


In [136]:
# Filter robo to rows where Transaction identifier is in check list
robo_matched = robo[robo['Transaction identifier'].isin(check['Transaction identifier'])]

## Make this table work for Moveo:

In [137]:
# Start with your renamed robo_matched
robo_matched = robo_matched.rename(columns={
    'Transaction identifier': 'roboticket_id__c',
    'Date': 'payment_date__c',
    'Products price': 'base_price__c',
    'Delivery price': 'delivery_price__c',
    'Total': 'price__c',
    'Payment type': 'payment_type__c',
    'Payment method': 'payment_method__c'
})

# Add new columns directly without overriding existing ones
robo_matched['Name'] = robo_matched['roboticket_id__c']
robo_matched['owner_account__r:Account:hjbc_id__c'] = None
robo_matched['_Cowner Account User ID'] = None

robo_matched['finish_date__c'] = robo_matched['payment_date__c']
robo_matched['created_at__c'] = robo_matched['payment_date__c']
robo_matched['updated_on__c'] = robo_matched['payment_date__c']

robo_matched['_Payment Gate'] = None
robo_matched['_Status'] = None
robo_matched['sales_man__c'] = None
robo_matched['_Created By'] = None

colunns_to_keep = [
    'roboticket_id__c',
    'Name',
    'owner_account__r:Account:hjbc_id__c',
    '_Cowner Account User ID',
    'Transaction owner email',
    'TransactionCoownerEmail',
    'finish_date__c',
    'created_at__c',
    'updated_on__c',
    'base_price__c',
    'delivery_price__c',
    'price__c',
    'payment_type__c',
    'payment_method__c',
    '_Payment Gate',
    '_Status',
    'sales_man__c',
    '_Created By'
]

transactions = robo_matched[colunns_to_keep]

transactions


,roboticket_id__c,Name,owner_account__r:Account:hjbc_id__c,_Cowner Account User ID,Transaction owner email,TransactionCoownerEmail,finish_date__c,created_at__c,updated_on__c,base_price__c,delivery_price__c,price__c,payment_type__c,payment_method__c,_Payment Gate,_Status,sales_man__c,_Created By
195025,3274214.0,3274214.0,None,None,jucha2014@gmail.com,NaN,2024-05-01 00:00:00,2024-05-01 00:00:00,2024-05-01 00:00:00,100.0,0.0,100.0,CashDesk,Cash,None,None,None,None
195026,3274215.0,3274215.0,None,None,jucha2014@gmail.com,NaN,2024-05-01 00:00:00,2024-05-01 00:00:00,2024-05-01 00:00:00,100.0,0.0,100.0,CashDesk,Cash,None,None,None,None
195027,3274216.0,3274216.0,None,None,jucha2014@gmail.com,NaN,2024-05-01 00:00:00,2024-05-01 00:00:00,2024-05-01 00:00:00,100.0,0.0,100.0,CashDesk,Cash,None,None,None,None
195028,3274217.0,3274217.0,None,None,jucha2014@gmail.com,NaN,2024-05-01 00:00:00,2024-05-01 00:00:00,2024-05-01 00:00:00,100.0,0.0,100.0,CashDesk,Cash,None,None,None,None
195029,3274218.0,3274218.0,None,None,yitzbasketball@gmail.com,NaN,2024-05-01 00:00:00,2024-05-01 00:00:00,2024-05-01 00:00:00,50.0,0.0,50.0,CashDesk,Cash,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195150,3274339.0,3274339.0,None,None,zuridan@gmail.com,NaN,2024-05-01 00:00:00,2024-05-01 00:00:00,2024-05-01 00:00:00,150.0,0.0,150.0,CashDesk,Cash,None,None,None,None
195151,3274340.0,3274340.0,None,None,NaN,NaN,2024-05-01 00:00:00,2024-05-01 00:00:00,2024-05-01 00:00:00,80.0,0.0,80.0,CashDesk,Cash,None,None,None,None
195152,3274341.0,3274341.0,None,None,NaN,NaN,2024-05-01 00:00:00,2024-05-01 00:00:00,2024-05-01 00:00:00,80.0,0.0,80.0,CashDesk,Cash,None,None,None,None
195153,3274342.0,3274342.0,None,None,neveragain9753@gmail.com,NaN,2024-05-01 00:00:00,2024-05-01 00:00:00,2024-05-01 00:00:00,100.0,0.0,100.0,CashDesk,Cash,None,None,None,None


In [139]:
# 1. Merge id_owner and id_coowner
users_clean = users[['email', 'userid']].drop_duplicates(subset='email')

transactions = transactions.merge(
    users_clean.rename(columns={'email': 'Transaction owner email', 'userid': 'id_owner'}),
    on='Transaction owner email',
    how='left'
)

transactions = transactions.merge(
    users_clean.rename(columns={'email': 'TransactionCoownerEmail', 'userid': 'id_coowner'}),
    on='TransactionCoownerEmail',
    how='left'
)

# 2. Define your functions
def resolve_sales_fields(row):
    # Case 1: both emails present
    if pd.notna(row.get('Transaction owner email')) and pd.notna(row.get('TransactionCoownerEmail')):
        return pd.Series({
            'sales_man__c': row['Transaction owner email'],
            'owner_account__r:Account:hjbc_id__c': row['id_coowner'],
            '_Cowner Account User ID': row['id_coowner']
        })

    # Case 2: only owner email present
    if pd.notna(row.get('Transaction owner email')) and pd.isna(row.get('TransactionCoownerEmail')):
        return pd.Series({
            'sales_man__c': None,
            'owner_account__r:Account:hjbc_id__c': row['id_owner'],
            '_Cowner Account User ID': None
        })

    # Case 3: no emails, but id_owner exists
    if pd.isna(row.get('Transaction owner email')) and pd.isna(row.get('TransactionCoownerEmail')) and pd.notna(row.get('id_owner')):
        return pd.Series({
            'sales_man__c': None,
            'owner_account__r:Account:hjbc_id__c': row['id_owner'],
            '_Cowner Account User ID': None
        })

    # Fallback: nothing found
    return pd.Series({
        'sales_man__c': None,
        'owner_account__r:Account:hjbc_id__c': None,
        '_Cowner Account User ID': None
    })


transactions[['sales_man__c', 'owner_account__r:Account:hjbc_id__c', '_Cowner Account User ID']] = transactions.apply(
    resolve_sales_fields, axis=1
)

# Map 'Payment type' strings to integer codes for system compatibility
conditions = [
    transactions['payment_type__c'] == 'Internet',
    transactions['payment_type__c'] == 'CashDesk',
    transactions['payment_type__c'] == 'Api'
]
choices = [1, 2, 4]
transactions['payment_type__c'] = np.select(conditions, choices, default=transactions['payment_type__c'])

transactions = transactions[colunns_to_keep]

transactions = transactions.drop(columns=['Transaction owner email', 'TransactionCoownerEmail'])

columns_to_clean = [
    'roboticket_id__c',
    'Name',
    'base_price__c',
    'delivery_price__c',
    'price__c'
]

for col in columns_to_clean:
    transactions[col] = pd.to_numeric(transactions[col], errors='coerce').apply(
        lambda x: int(x) if pd.notna(x) else None
    )

columns_to_clean = [
    'owner_account__r:Account:hjbc_id__c',
    '_Cowner Account User ID'
]

for col in columns_to_clean:
    transactions[col] = transactions[col].apply(
        lambda x: str(int(x)) if pd.notna(x) else None
    )

transactions

,roboticket_id__c,Name,owner_account__r:Account:hjbc_id__c,_Cowner Account User ID,finish_date__c,created_at__c,updated_on__c,base_price__c,delivery_price__c,price__c,payment_type__c,payment_method__c,_Payment Gate,_Status,sales_man__c,_Created By
0,3274214,3274214,10712,None,2024-05-01 00:00:00,2024-05-01 00:00:00,2024-05-01 00:00:00,100,0,100,2,Cash,None,None,NaN,None
1,3274215,3274215,10712,None,2024-05-01 00:00:00,2024-05-01 00:00:00,2024-05-01 00:00:00,100,0,100,2,Cash,None,None,NaN,None
2,3274216,3274216,10712,None,2024-05-01 00:00:00,2024-05-01 00:00:00,2024-05-01 00:00:00,100,0,100,2,Cash,None,None,NaN,None
3,3274217,3274217,10712,None,2024-05-01 00:00:00,2024-05-01 00:00:00,2024-05-01 00:00:00,100,0,100,2,Cash,None,None,NaN,None
4,3274218,3274218,10903,None,2024-05-01 00:00:00,2024-05-01 00:00:00,2024-05-01 00:00:00,50,0,50,2,Cash,None,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,3274339,3274339,48863,None,2024-05-01 00:00:00,2024-05-01 00:00:00,2024-05-01 00:00:00,150,0,150,2,Cash,None,None,NaN,None
126,3274340,3274340,18009,None,2024-05-01 00:00:00,2024-05-01 00:00:00,2024-05-01 00:00:00,80,0,80,2,Cash,None,None,NaN,None
127,3274341,3274341,18009,None,2024-05-01 00:00:00,2024-05-01 00:00:00,2024-05-01 00:00:00,80,0,80,2,Cash,None,None,NaN,None
128,3274342,3274342,48866,None,2024-05-01 00:00:00,2024-05-01 00:00:00,2024-05-01 00:00:00,100,0,100,2,Cash,None,None,NaN,None


In [140]:
transactions[transactions['roboticket_id__c'] == 3274242]

,roboticket_id__c,Name,owner_account__r:Account:hjbc_id__c,_Cowner Account User ID,finish_date__c,created_at__c,updated_on__c,base_price__c,delivery_price__c,price__c,payment_type__c,payment_method__c,_Payment Gate,_Status,sales_man__c,_Created By
28,3274242,3274242,18009,None,2024-05-01 00:00:00,2024-05-01 00:00:00,2024-05-01 00:00:00,50,0,50,2,Cash,None,None,NaN,None


In [141]:
transactions.to_excel('transactions_that_didnt_pass.xlsx', index=False)